In [ ]:
import os
os.getcwd()

In [ ]:
# Make sure the source code auto reloads into the kernel
%load_ext autoreload
%autoreload 2

In [ ]:
# To help preventing shared maemory errors
!ulimit -n 500000
import torch.multiprocessing
torch.multiprocessing.set_sharing_strategy('file_system')

In [ ]:
import random
from Utils.logger import logger
from Utils.train_and_eval import test_model
from Utils.train_and_eval import create_config

In [ ]:
# Model Configurations
random.seed(7)

test_websites = ['movie-boxofficemojo']
attributes = ['title', 'director', 'genre', 'mpaa_rating']

In [ ]:
config = create_config(train_websites=[], val_websites=[], test_websites=test_websites, attributes=attributes)
logger.info(f'The created model config is:\n{config}')

In [ ]:
logger.info(f'Start testing based on the validation data set')
test_model(config, model_weights_file=f'data/weights.ckpt')